In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, AveragePooling2D
from keras.utils.np_utils import to_categorical

Using Theano backend.
Using gpu device 0: Quadro K620 (CNMeM is disabled, cuDNN not available)


In [2]:
train=open("Downloads/Gun_Point_TRAIN")
test=open("Downloads/Gun_Point_TEST")
train = train.read().split()
test = test.read().split()

In [3]:
train = np.array_split(np.array(train),50)
test = np.array_split(np.array(test),150)
print(np.shape(train))
print(np.shape(test))

(50, 151)
(150, 151)


In [4]:
#get the label of train and test
def Get_label(data):
    label=[]
    for i in range(0,len(data)):
        label.append(float(data[i][0]))
    return label
y_train = Get_label(train)
y_test= Get_label(test)
print('train labels = ',y_train)

('train labels = ', [2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0])


In [5]:
#get the value of train and test
def Get_value(data):
    value=[]
    for i in range(0,len(data)):
        value.append(data[i][1:151])
    return value
X_train=Get_value(train)
X_test=Get_value(test)
print('shape:',np.shape(X_test))
print('shape:',np.shape(X_train))

('shape:', (150, 150))
('shape:', (50, 150))


In [7]:
#set the series 
length=150
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = np.asarray(y_train)
y_train= y_train.astype(int)
y_test = np.asarray(y_test)
y_test= y_test.astype(int)
y_test=y_test-1
y_train=y_train-1
X_train=X_train.reshape(50,1,150,1)
X_test=X_test.reshape(150,1,150,1)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

(50, 'train sequences')
(150, 'test sequences')
('X_train shape:', (50, 1, 150, 1))
('X_test shape:', (150, 1, 150, 1))


In [8]:
def process_output(labels):
    return to_categorical(labels, nb_classes=2)
y_train = process_output(y_train)
y_test = process_output(y_test)

In [9]:
print('Building model...')
model = Sequential()
#1st convolution
model.add(Convolution2D(50,8,1,input_shape=(1, 150, 1)))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2,1)))
#2st convolution
model.add(Convolution2D(100,8,1))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2,1)))
model.add(Flatten())
#fulley connected layer
model.add(Dense(500,activation="relu"))
model.add(Dense(2))
#softmax
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

Building model...


In [10]:
batch_size = 50
nb_epoch = 10

model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, y_test))

Train on 50 samples, validate on 150 samples
Epoch 1/10
50/50 [==============================] - 2s - loss: 0.7123 - acc: 0.5200 - val_loss: 0.9626 - val_acc: 0.5067
Epoch 2/10
50/50 [==============================] - 2s - loss: 0.9636 - acc: 0.4800 - val_loss: 1.3310 - val_acc: 0.4933
Epoch 3/10
50/50 [==============================] - 2s - loss: 1.2333 - acc: 0.5200 - val_loss: 0.8368 - val_acc: 0.5067
Epoch 4/10
50/50 [==============================] - 2s - loss: 0.8232 - acc: 0.4800 - val_loss: 0.6755 - val_acc: 0.4933
Epoch 5/10
50/50 [==============================] - 2s - loss: 0.5903 - acc: 0.5200 - val_loss: 0.5747 - val_acc: 0.6800
Epoch 6/10
50/50 [==============================] - 2s - loss: 0.5148 - acc: 0.7400 - val_loss: 0.5636 - val_acc: 0.6600
Epoch 7/10
50/50 [==============================] - 2s - loss: 0.4568 - acc: 0.7400 - val_loss: 0.5162 - val_acc: 0.7267
Epoch 8/10
50/50 [==============================] - 2s - loss: 0.4253 - acc: 0.8400 - val_loss: 0.5283 - val

In [11]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 50, 143, 1)  450         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 50, 143, 1)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
averagepooling2d_1 (AveragePooling2(None, 50, 71, 1)   0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 100, 64, 1)  40100       averagepooling2d_1[0][0]         
___________________________________________________________________________________________

In [12]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('Test score:', 0.51452524344126382)
('Test accuracy:', 0.6999999968210856)
